In [15]:
# Through brainstorming, I realized we need a clocked simulator. Here it is.
import time

In [41]:
ambulance_count = 8

# I need ambulance enums

amb_status = {}

for ID in range(0, ambulance_count):
    amb_status[ID] = "idle"

In [39]:
def timed_execute():
    """ Starts a clock """
    speed_times = 4
    run_time = 5
    start = time.time()
    time_diff = 0

    while True:
        """ This code makes the world run over time. """
        time.sleep(1/speed_times)
        
        print ("Time: " + str(time_diff))
        change_state()
        print()
        
        now = time.time()

        time_diff = (now - start)*speed_times
        time_diff = int(time_diff)
        
        if time_diff > run_time: break 

In [40]:
timed_execute()

Time: 0

Time: 1

Time: 2

Time: 3

Time: 4

Time: 5



In [21]:
def change_state():
    global amb_status
    
    for ID in amb_status:
        
    
    pass